In [1]:
import numpy as np
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Agrupación (groupby)

## Lo Básico

In [8]:
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                    'key2': ['one','two','one','two','one'],
                    'data1': np.random.randn(5),
                    'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.496714,-0.234137
1,a,two,-0.138264,1.579213
2,b,one,0.647689,0.767435
3,b,two,1.523030,-0.469474
4,a,one,-0.234153,0.542560


In [9]:
# Calcula la media por cada grupo
grouped = df['data1'].groupby(df['key1']) # La columna llave será 'data'
grouped # Objeto GroupBy
grouped.mean() # Calcula media

key1
a    0.041432
b    1.085359
Name: data1, dtype: float64

In [11]:
# Pasar arrays múltiples arrojará un objeto groupBy jerarquizado
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means # Nested grouping

key1  key2
a     one     0.131280
      two    -0.138264
b     one     0.647689
      two     1.523030
Name: data1, dtype: float64

In [13]:
# Podemos convertir ese objeto Series jerarquizado en un DF
means.unstack()

key2,one,two
key1,,
a,0.131280,-0.138264
b,0.647689,1.523030


In [14]:
# Agrupando por columnas criterio externas y diferentes
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
# Cada combinación de valores entre las llaves se toma como identificador
df['data1'].groupby([states, years]).mean()

California  2005   -0.138264
            2006    0.647689
Ohio        2005    1.009872
            2006   -0.234153
Name: data1, dtype: float64

In [ ]:
# Si la llave (columna) de agrupación ya está en el DF, no necesitamos declararla
df.groupby('key1').mean() # Nótese que 'key 2' no aparece (nuisance column, datos no numéricos)
df.groupby(['key1', 'key2']).mean()

In [15]:
# Un uso común de la agrupación es calcular el tamaño de grupos
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterando sobre grupos

In [17]:
# Generemos los grupos uno por uno (primero nombre, luego valores)
df
for name, group in df.groupby('key1'):
    print(name)
    print(group)

,key1,key2,data1,data2
0,a,one,0.496714,-0.234137
1,a,two,-0.138264,1.579213
2,b,one,0.647689,0.767435
3,b,two,1.523030,-0.469474
4,a,one,-0.234153,0.542560


a
  key1 key2     data1     data2
0    a  one  0.496714 -0.234137
1    a  two -0.138264  1.579213
4    a  one -0.234153  0.542560
b
  key1 key2     data1     data2
2    b  one  0.647689  0.767435
3    b  two  1.523030 -0.469474


In [24]:
# Para agrupaciones por llave múltiple, el resultado de iterar es una tupla
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2)) # tupla(tupla de combinación, valores)
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.496714 -0.234137
4    a  one -0.234153  0.542560
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.138264  1.579213
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.647689  0.767435
('b', 'two')
  key1 key2    data1     data2
3    b  two  1.52303 -0.469474


In [ ]:
# Convertir objeto GroupBy a diccionario y acceder a los grupos
pieces = dict(list(df.groupby('key1')))
pieces
pieces['b']

In [26]:
# Agrupando a lo largo de columnas
df.dtypes # Agrupando por tipo de datos (object, float64)
grouped = df.groupby(df.dtypes, axis=1)
# Veamos los grupos
for dtype, group in grouped:
    print(dtype)
    print(group)

key1      object
key2      object
data1    float64
data2    float64
dtype: object

float64
      data1     data2
0  0.496714 -0.234137
1 -0.138264  1.579213
2  0.647689  0.767435
3  1.523030 -0.469474
4 -0.234153  0.542560
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## Selección

In [29]:
# Azúcar sintáctica para df['data1'].groupby(df['key1'])
df.groupby('key1')['data1'] # Obj GrpBy como Series
df.groupby('key1')[['data2']] # Obj GrpBy como DF

In [30]:
# Resulta útil para revisar sólo una columna
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.154212
     two   1.579213
b    one   0.767435
     two  -0.469474

In [33]:
# Obtenemos un Df si selccionamos dos o más columnas para revisar
s_grouped = df.groupby(['key1', 'key2'])['data2'] # Series jerarquizado
d_grouped = df.groupby(['key1', 'key2'])['data1', 'data2'] # DF
s_grouped
d_grouped
s_grouped.mean()
d_grouped.mean()

key1  key2
a     one     0.154212
      two     1.579213
b     one     0.767435
      two    -0.469474
Name: data2, dtype: float64

data1     data2
key1 key2                    
a    one   0.131280  0.154212
     two  -0.138264  1.579213
b    one   0.647689  0.767435
     two   1.523030 -0.469474

## Agrupando con diccionarios

In [34]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a','b','c','d','e'],
                     index=['Joe','Steve','Wes','Jim','Travis'])

In [36]:
# Añadimos unos cuentos valores nulos
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-0.463418,-0.465730,0.241962,-1.913280,-1.724918
Steve,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
Wes,1.465649,NaN,NaN,-1.424748,-0.544383
Jim,0.110923,-1.150994,0.375698,-0.600639,-0.291694
Travis,-0.601707,1.852278,-0.013497,-1.057711,0.822545


In [37]:
# Secuencia grupo-valores
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [39]:
# Método eficiente si ya tenemos los valores únicos de la columna criterio
by_column = people.groupby(mapping, axis=1)
by_column.sum() # Suma a lo largo de las columnas (axis=1)

,blue,red
Joe,-1.671318,-2.654065
Steve,-0.593777,-2.987422
Wes,-1.424748,0.921266
Jim,-0.224941,-1.331765
Travis,-1.071208,2.073116


In [40]:
# En lugar de un diccionario, podemos utilizar un Series
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Agrupando con funciones

In [41]:
# Agrupación basada en el largo de los valores de columnas clave
# Aplicamos len() a cada valor y utilizamos el entero como valor criterio
people
people.groupby(len).sum() 

,a,b,c,d,e
Joe,-0.463418,-0.465730,0.241962,-1.913280,-1.724918
Steve,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
Wes,1.465649,NaN,NaN,-1.424748,-0.544383
Jim,0.110923,-1.150994,0.375698,-0.600639,-0.291694
Travis,-0.601707,1.852278,-0.013497,-1.057711,0.822545


,a,b,c,d,e
3,1.113154,-1.616723,0.617660,-3.938667,-2.560994
5,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
6,-0.601707,1.852278,-0.013497,-1.057711,0.822545


In [42]:
# Un ejemplo más difícil
key_list = ['one','one','one','two','two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.463418 -0.465730  0.241962 -1.913280 -1.724918
  two  0.110923 -1.150994  0.375698 -0.600639 -0.291694
5 one -0.562288 -1.012831  0.314247 -0.908024 -1.412304
6 two -0.601707  1.852278 -0.013497 -1.057711  0.822545

# Agregaciones (agg)

#### Aggregations: Tranformación que convierta arrays en números (escalares)
- count: ¿Cuántos existen?
- sum: ¿Cuánto representan?
- mean: Media
- median: Mediana
- std, var: Desv. est., varianza
- min, max: Mínimos  máximos
- prod: Product de valores no nulos
- first, last: Primer o último valore

Cada una de estas funciones ignora los valores NaN y tiene implmentaciones optimizadas

## Lo Básico

In [45]:
# Podemos usar funciones que no estén definidos en un GrpObj, pero sí en otras variables
# quantile() está definido en Series, no en GroupBy
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9) # Posible valor en posicion 90 del rango 

,key1,key2,data1,data2
0,a,one,0.496714,-0.234137
1,a,two,-0.138264,1.579213
2,b,one,0.647689,0.767435
3,b,two,1.523030,-0.469474
4,a,one,-0.234153,0.542560


key1
a    0.369718
b    1.435496
Name: data1, dtype: float64

In [46]:
# No es una agregación, pero ilustra el uso de funciones no definidas
grouped.describe()

data1                                                             \
     count      mean      std       min       25%       50%       75%   
key1                                                                    
a      3.0  0.041432  0.39719 -0.234153 -0.186209 -0.138264  0.179225   
b      2.0  1.085359  0.61896  0.647689  0.866524  1.085359  1.304195   

               data2                                                   \
           max count      mean       std       min       25%      50%   
key1                                                                    
a     0.496714   3.0  0.629212  0.909775 -0.234137  0.154212  0.54256   
b     1.523030   2.0  0.148980  0.874627 -0.469474 -0.160247  0.14898   

                          
           75%       max  
key1                      
a     1.060886  1.579213  
b     0.458207  0.767435

In [44]:
# Agregamos usando agg(). Podemos agregar pasando nuestras propies funciones incluso
def peak_to_peak(arr):
    return arr.max() - arr.min()

# Llama función personalizada con agg()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.730868,1.813350
b,0.875341,1.236909


## Mapeo de Agregaciones

## apply() en objetos GroupBy